# Perceptron 

In this lab I will apply the Perceptron algorithm to some classification tasks. A simple implementation of the Perceptron algorithm and its application is provided. The main purpose of this session is to extend the example given to other tasks, trying to minimize test error.

# Perceptron applied to the Iris dataset

**Reading the dataset:** $\;$ I also check that the data matrix and labels have the right number of rows and columns

In [2]:
import numpy as np;
from sklearn.datasets import load_iris
iris = load_iris() # Charge le dataset Iris
X = iris.data.astype(np.float16) # Convertit les caractéristiques (features) en type float16 pour économiser de la mémoire.
y = iris.target.astype(np.uint).reshape(-1, 1) # Convertit les cibles en entiers non signés (uint) et les redimensionne pour être une matrice colonne

print(X.shape, y.shape, "\n", np.hstack([X, y])[:5, :])  # Combine horizontalement les caractéristiques X et les cibles y
# Affiche les 5 premières lignes de cette matrice 

(150, 4) (150, 1) 
 [[5.1015625  3.5        1.40039062 0.19995117 0.        ]
 [4.8984375  3.         1.40039062 0.19995117 0.        ]
 [4.69921875 3.19921875 1.29980469 0.19995117 0.        ]
 [4.6015625  3.09960938 1.5        0.19995117 0.        ]
 [5.         3.59960938 1.40039062 0.19995117 0.        ]]


**Dataset partition:** $\;$ I create a split of the Iris dataset with $20\%$ of data for test and the rest for training, previously shuffling the data according to a given seed provided by a random number generator. Here, as in all code that includes randomness (which requires generating random numbers), it is convenient to fix said seed to be able to reproduce experiments with accuracy.

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
# Fonction utilisée pour diviser les données en deux sous-ensembles : un pour l'entraînement et un pour les tests

print(X_train.shape, X_test.shape)

(120, 4) (30, 4)


**Perceptron implementation:** $\;$ returns weights in homogeneous notation, $\mathbf{W}\in\mathbb{R}^{(1+D)\times C};\;$ also the number of errors and iterations executed

In [18]:
import numpy as np;
def perceptron(X, y, b=0.1, a=1.0, K=200):
    """
    Perceptron multi-classes.
    
    X : array (N, D), données d'entrée (N exemples, D caractéristiques)
    y : array (N,), étiquettes de classe
    b : float, biais pour le critère d'activation (par défaut 0.1)
    a : float, taux d'apprentissage (par défaut 1.0)
    K : int, nombre maximal d'itérations (par défaut 200)
    
    Retourne :
    W : array (D+1, C), matrice des poids
    E : int, nombre d'erreurs lors de la dernière itération
    k : int, nombre d'itérations effectuées
    """
    
    # Initialisation des dimensions et paramètres
    N, D = X.shape  # Nombre d'exemples (N) et de caractéristiques (D)
    
    Y = np.unique(y)  # Classes uniques dans y, supprime les doublons pour pouvoir savoir combien de classes on a 
    #parce que y est un vecteur qui contient a chaque position n la classe de la donnée n
    
    C = Y.size  # Nombre total de classes
    W = np.zeros((1 + D, C))  # Matrice des poids, initialisée à zéro

    # Boucle principale pour un maximum de K itérations et a chaque fois on itère sur tte la data 
    for k in range(1, K + 1):
        E = 0  # Compteur d'erreurs pour cette itération

        # Boucle sur chaque exemple
        for n in range(N):
            # Ajout d'un biais (1) à chaque ligne n au fur et a mesure que l'on parcourt les N données  et xn c est une seule ligne avec le biais 1 au debut 
            xn = np.array([1, *X[n, :]])                  
            # Identifie la classe correcte pour cet exemple = one hot encoding 
            cn = np.squeeze(np.where(Y == y[n])) #bhala db ila eadna Y=[1,2,3] o y[n]=2 alors cn=[0,1,0]
            # Calcul de la sortie pour la classe correcte
            gn = W[:, cn].T @ xn #ca c est la classe prédite avec les current weights genre on prend juste la ligne des weights qui correspond à la classe correcte et on la multiplie avec la ligne de la donnée
            # Indicateur d'erreur
            err = False

            # On trouve gn de toutes les autres classes avec les weights qu on a 
            for c in np.arange(C):
                # On check si gn calculé avec la classe correcte est la plus grande et donc c'est celle qui va etre predite  avec les current weights 
                if c != cn and W[:, c].T @ xn + b >= gn:
                    W[:, c] = W[:, c] - a * xn  # Diminue les poids pour la classe incorrecte
                    err = True  # Une erreur est détectée
            
            # Si une erreur a été détectée, met à jour la classe correcte
            if err:
                W[:, cn] = W[:, cn] + a * xn  # Augmente les poids pour la classe correcte
                E = E + 1  # Incrémente le compteur d'erreurs
        
        # Si aucune erreur pour tout les donnees N, cela veut dire qu ils sont tous bien classés avec les current weights, arrête l'entraînement (convergence atteinte) 
        if E == 0:
            break

    # Retourne la matrice des poids, le nombre d'erreurs et d'itérations effectuées
    return W, E, k


**Learning a (linear) classifier with Perceptron:** $\;$ Perceptron minimizes the number of training errors (with margin $b$)
$$\mathbf{W}^*=\operatorname*{argmin}_{\mathbf{W}=(\boldsymbol{w}_1,\dotsc,\boldsymbol{w}_C)}\sum_n\;\mathbb{ I}\biggl(\max_{c\neq y_n}\;\boldsymbol{w}_c^t\boldsymbol{x}_n+b \;>\; \boldsymbol{w}_{y_n}^t\boldsymbol{ x}_n\biggr)$$

In [19]:
#Dans ce cas la, ca s est pas naturellement arrete donc ca a juste atteint le nombre max d iterations et du cp ca a donne les weights de la dernière iteration 
W, E, k = perceptron(X_train, y_train) 
print("Number of iterations executed: ", k)
print("Number of training errors: ", E)
print("Weight vectors of the classes (in columns and with homogeneous notation):\n", W)

Number of iterations executed:  200
Number of training errors:  2
Weight vectors of the classes (in columns and with homogeneous notation):
 [[  10.           85.         -142.        ]
 [ -49.421875    -68.19140625 -176.47265625]
 [  50.171875     -1.72460938 -181.06445312]
 [-189.91210938  -87.70507812   68.69726562]
 [ -86.40258789 -137.78149414  157.88415527]]


<p style="page-break-after:always;"></p>

**Calculation of test error rate:**

In [5]:
X_testh = np.hstack([np.ones((len(X_test), 1)), X_test]) # Combine une colonne de 1 (biais) à X_test
y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1) # Calcule les prédictions pour l'ensemble de test = trouve l'indice (classe) avec le score maximal pour chaque exemple.
err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test) # Calcule le taux d'erreur sur l'ensemble de test
print(f"Error rate on test: {err_test:.1%}")

Error rate on test: 16.7%


**Margin adjustment:** $\;$ experiment to learn a value of $b$

In [6]:
# Teste le perceptron avec différents biais (b) et évalue ses performances
for b in (.0, .01, .1, 10, 100): # Liste de valeurs pour le biais
    W, E, k = perceptron(X_train, y_train, b=b, K=1000) # Entraîne le perceptron avec le biais b, un maximum de 1000 itérations
    print(b, E, k) 

0.0 3 1000
0.01 5 1000
0.1 3 1000
10 6 1000
100 6 1000


**Interpretation of results:** $\;$ the training data does not appear to be linearly separable; it is not clear that a margin greater than zero can improve results, especially since we only have $30$ test samples; with a margin $b=0.1$ we have already seen that an error (in test) of $16.7\%$ is obtained